# Milestone #2: Ravi Raghavan

## Milestone Aim

The goal is to use a **pretrained BERT-Base, Uncased** model and **fine-tune it on the r/Fakeeddit dataset**.

This work presents evaluation results on: 
- Pretrained BERT
- Fine-Tuned BERT


## Script Sanity Check

Please ensure your directory is structured as follows

```text
cleaned_data/
├── test_5k.csv
├── train.csv
└── validation_5k.csv

## Environment Setup

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import sys
import torch.nn as nn

## Ensure TensorFlow is not used
import os
os.environ["USE_TF"] = "0"

# Import Hugging Face Tooling
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import evaluate
from datasets import Dataset

# Define data directory
DATA_DIR = "cleaned_data"

# Define file paths
TRAIN_DATA_FILE = os.path.join(DATA_DIR, "train.csv")
VALIDATION_DATA_FILE = os.path.join(DATA_DIR, "validation_5k.csv")
TEST_DATA_FILE = os.path.join(DATA_DIR, "test_5k.csv")

# For reproducability
random_state = 42

# Use CPU/MPS if possible
device = None
if "google.colab" in sys.modules:
    # Running in Colab
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
else:
    # Not in Colab (e.g., Mac)
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

print("Using device:", device)

Using device: mps


## Load Data

In [2]:
TRAIN_DATA = pd.read_csv(TRAIN_DATA_FILE)
VALIDATION_DATA = pd.read_csv(VALIDATION_DATA_FILE, index_col = 0)
TEST_DATA = pd.read_csv(TEST_DATA_FILE, index_col = 0)

In [3]:
# Ignore rows in corrupted_indices.txt files
def filter_out_corrupted_rows(split, DF):
    # File with corrupted indices
    if split == "train":
        corrupted_indices_file = f"{split}_corrupted_indices.txt"
    else:
        corrupted_indices_file = f"{split}_5k_corrupted_indices.txt"

    # Store list of corrupted indices
    corrupted_indices = None

    # Get list of corrupted indices
    with open(corrupted_indices_file, "r") as f:
        corrupted_indices = list(int(line.strip()) for line in f if line.strip())

    print(f"Split: {split}, Corrupted Indices: {corrupted_indices}, Length: {len(corrupted_indices)}")

    # Filter out corrupted rows
    DF = DF.drop(index = corrupted_indices)

    return DF

In [4]:
TRAIN_DATA = filter_out_corrupted_rows("train", TRAIN_DATA)
VALIDATION_DATA = filter_out_corrupted_rows("validation", VALIDATION_DATA)
TEST_DATA = filter_out_corrupted_rows("test", TEST_DATA)

Split: train, Corrupted Indices: [2862, 26040, 28337, 18547, 13374, 11288, 31984, 18451, 19000, 22479, 8048, 32075, 22918, 5586, 19345, 12770, 32189, 14628, 9081, 6611, 2927], Length: 21
Split: validation, Corrupted Indices: [6568, 32176], Length: 2
Split: test, Corrupted Indices: [29133, 9437, 26504, 11394], Length: 4


In [5]:
TRAIN_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
0,this spongebob squarepants branded battery,2019-07-30 20:00:50,i.redd.it,https://preview.redd.it/f39wxxk8yhd31.jpg?widt...,4.0,33,mildlyinteresting,0.95,1,0,0
1,award for careless talk,2011-09-03 17:26:23,i.imgur.com,https://external-preview.redd.it/KgPHCi1u3fY5j...,1.0,14,propagandaposters,1.00,0,1,5
2,four aligned airplanes,2017-11-20 06:05:45,i.redd.it,https://preview.redd.it/88v9axk19phx.jpg?width...,24.0,198,confusing_perspective,0.98,0,2,2
3,columbus discovers the new world,2019-08-28 15:40:17,i.redd.it,https://preview.redd.it/x4wzpd0am7j31.jpg?widt...,5.0,318,fakehistoryporn,0.98,0,2,2
4,feed me drummmmssssssss,2014-05-09 13:23:59,i.imgur.com,https://external-preview.redd.it/yNN57loQnVhLk...,0.0,3,pareidolia,0.62,0,2,2


In [6]:
VALIDATION_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
8637,not as heartwarming as it could have been anth...,2019-09-19 17:48:33,lifestyle.clickhole.com,https://external-preview.redd.it/850kBbKdgMKfz...,0.0,15,theonion,0.86,0,2,1
20669,other discussions,2013-12-09 23:58:43,NaN,http://i.dailymail.co.uk/i/pix/2013/12/09/arti...,NaN,0,psbattle_artwork,NaN,0,2,4
13179,on reflection oc,2015-01-12 17:14:55,i.imgur.com,https://external-preview.redd.it/tiFw8Ggb178E4...,0.0,3,pareidolia,0.67,0,2,2
20565,viet congo setting booby trap,2019-02-27 09:57:07,i.redd.it,https://preview.redd.it/mj81gkh533j21.jpg?widt...,4.0,14,fakehistoryporn,0.80,0,2,2
15504,chief has some happy shoulder armour,2013-02-20 01:00:02,i.imgur.com,https://external-preview.redd.it/kqqIlnAOZxzxA...,0.0,6,pareidolia,1.00,0,2,2


In [7]:
TEST_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
19660,young homosexuals gather outside of a nightclu...,2018-07-29 13:59:38,i.redd.it,https://preview.redd.it/16j830998wc11.jpg?widt...,1.0,75,fakehistoryporn,0.92,0,2,2
32879,cara al sol facing the sun a series of posters...,2013-05-20 18:25:00,imgur.com,https://external-preview.redd.it/S_nXt5X8VMqZD...,4.0,19,propagandaposters,0.89,0,1,5
15071,he is awake and rises from the depths,2013-10-13 20:02:17,imgur.com,https://external-preview.redd.it/JetvyFQFm4fYt...,0.0,7,pareidolia,0.99,0,2,2
5198,the reason germany invades france and not spain,2018-09-03 08:22:07,i.redd.it,https://preview.redd.it/ws91cs7lgzj11.png?widt...,0.0,57,fakehistoryporn,0.97,0,2,2
15061,frozen body preserved on mount everest nsfw,2017-09-12 08:10:21,i.redd.it,https://preview.redd.it/zxwc8gq8uelz.jpg?width...,0.0,24,fakehistoryporn,1.00,0,2,2


## Compute Class Proportions

In [8]:
# Compute Class Proportions
p0 = (TRAIN_DATA['2_way_label'] == 0).mean() # Computes the percentage of our training dataset that has label = 0 [Fake News]
p1 = (TRAIN_DATA['2_way_label'] == 1).mean() # Computes the percentage of our training dataset that has label = 1 [Non-Fake News]
print(f"{p0  * 100}% of our dataset has label = 0 and {p1  * 100}% of our dataset has label = 1")

44.62961294778248% of our dataset has label = 0 and 55.37038705221752% of our dataset has label = 1


## Define Prior Adjusted Loss Criterion

In [9]:
# Define Weighted Loss Criterion
class_weights = torch.tensor([p1, p0]).float().to(device)
custom_criterion = nn.CrossEntropyLoss(weight = class_weights)
print(f"Class Weights: {class_weights}")

Class Weights: tensor([0.5537, 0.4463], device='mps:0')


## Fetch BERT From HuggingFace

In [10]:
# Fetch BERT Model from HuggingFace
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels = 2) # num_labels = 2 since we have 2 classes!

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create `Hugging Face` Datasets [Train + Dev + Test]


In [11]:
train_hf_dataset = Dataset.from_pandas(TRAIN_DATA)
dev_hf_dataset = Dataset.from_pandas(VALIDATION_DATA)
test_hf_dataset = Dataset.from_pandas(TEST_DATA)

## Tokenize Text Data

In [12]:
def tokenize_function(row):
  tokens = tokenizer(row['clean_title'], truncation = True, padding = 'max_length', max_length = tokenizer.model_max_length)
  row['input_ids'] = tokens['input_ids']
  row['attention_mask'] = tokens['attention_mask']
  row['token_type_ids'] = tokens['token_type_ids']
  row['label'] = int(row['2_way_label'])
  return row

In [13]:
train_hf_dataset = train_hf_dataset.map(tokenize_function)
dev_hf_dataset = dev_hf_dataset.map(tokenize_function)
test_hf_dataset = test_hf_dataset.map(tokenize_function)

Map:   0%|          | 0/33303 [00:00<?, ? examples/s]

Map:   0%|          | 0/4998 [00:00<?, ? examples/s]

Map:   0%|          | 0/4996 [00:00<?, ? examples/s]

## Define Accuracy, Precision, Recall, and F1 Metrics from Hugging Face

In [14]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load("f1")

## Define a compute_metrics function

In [15]:
def compute_metrics(eval_pred):
    # Get the model predictions
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Return Metrics
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'], # Accuracy
        "pos_precision": precision_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary', zero_division = 0)["precision"], # Precision on the Class w/ Label = 1 [Hate Samples]
        "pos_recall": recall_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary', zero_division = 0)['recall'], # Recall on the Class w/ Label = 1 [Hate Samples]
        "pos_f1": f1_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary')["f1"], # F1 Score on the Class w/ Label = 1 [Hate Samples]
        "neg_precision": precision_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary', zero_division = 0)['precision'], # Precision on the Class w/ Label = 0 [Non-Hate Samples]
        "neg_recall": recall_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary', zero_division = 0)['recall'], # Recall on the Class w/ Label = 0 [Non-Hate Samples]
        "neg_f1": f1_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary')['f1'], # F1 Score on the Class w/ Label = 0 [Non-Hate Samples]
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1'], # Macro F1 Score
        "f1_micro": f1_metric.compute(predictions=predictions, references=labels, average='micro')['f1'], # Micro F1 Score
        "f1_weighted": f1_metric.compute(predictions=predictions, references=labels, average='weighted')['f1'], # Weighted F1 Score
    }


## Subclass the `Trainer` Class from HuggingFace to use Custom Loss Criterion


In [16]:
# Create a subclassed Trainer that enables us to use the custom loss function defined earlier
class SubTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs = False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_criterion(logits, labels)
        return (loss, outputs) if return_outputs else loss

## **Initialize the `TrainingArguments` and `Trainer`**

In [17]:
training_args = TrainingArguments(
    output_dir="Milestone2-Baseline-BERT-FineTuning",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_strategy="steps",      # save checkpoints every N steps
    save_steps=100,             # save every 100 steps
    eval_strategy="steps",      # evaluate every N steps
    eval_steps=100,             # evaluate every 100 steps
    logging_strategy="steps",
    logging_steps=100,          # log every 100 steps
    report_to="none",
    full_determinism=True
)

trainer = SubTrainer(
    model=model,
    args=training_args,
    train_dataset=train_hf_dataset,
    eval_dataset=dev_hf_dataset,
    compute_metrics=compute_metrics,
)

# **Evaluate Pre-Trained Model on Train, Dev, and Test Datasets**

In [18]:
# Split: Train, Dev, or Test
def generate_evaluation_results(split):
    dataset = None
    if split == "train":
        dataset = train_hf_dataset
    elif split == "dev" or split == "validation" or split == "val":
        dataset = dev_hf_dataset
    elif split == "test":
        dataset = test_hf_dataset
    
    results = trainer.evaluate(eval_dataset=dataset, metric_key_prefix=split)
    df_results = pd.DataFrame([results])
    df_results.to_csv(f"Milestone #2 Pre-Trained BERT Baseline {split} Results.csv", index=False)
    print(f"Saved {split} evaluation metrics to Milestone #2 Pre-Trained BERT Baseline {split} Results.csv")

# Generate Evaluation Results on Train, Dev, and Test Splits
generate_evaluation_results("train")
generate_evaluation_results("dev")
generate_evaluation_results("test")

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved train evaluation metrics to Milestone #2 Pre-Trained BERT Baseline train Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved dev evaluation metrics to Milestone #2 Pre-Trained BERT Baseline dev Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved test evaluation metrics to Milestone #2 Pre-Trained BERT Baseline test Results.csv


# **Train the Model: `Fine-Tuning`**

In [19]:
trainer.train() # Always Resume from Last Checkpoint to Save Time
trainer.save_model('Milestone2-Baseline-BERT-FinalModel') # Save the Final Model
trainer.save_state() # Save the State of the Trainer (e.g. Losses, etc)

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Pos Precision,Pos Recall,Pos F1,Neg Precision,Neg Recall,Neg F1,F1 Macro,F1 Micro,F1 Weighted
100,0.575700,0.487445,0.000900,0.770908,0.750926,0.878022,0.809516,0.807736,0.637629,0.712673,0.761094,0.770908,0.766365
200,0.493000,0.471866,0.000900,0.791317,0.839089,0.771563,0.803911,0.741633,0.815896,0.776994,0.790452,0.791317,0.791917
300,0.460400,0.454135,0.000900,0.804722,0.807258,0.850956,0.828531,0.801156,0.747194,0.773234,0.800883,0.804722,0.803892
400,0.448000,0.431254,0.000900,0.799720,0.844626,0.782750,0.812512,0.752263,0.820835,0.785055,0.798783,0.799720,0.800277
500,0.464500,0.416621,0.000900,0.810524,0.802588,0.872970,0.836301,0.822581,0.732824,0.775113,0.805707,0.810524,0.809037
600,0.437300,0.412407,0.000900,0.809724,0.777608,0.919885,0.842784,0.870930,0.672654,0.759058,0.800921,0.809724,0.805477
700,0.421800,0.392265,0.000900,0.826731,0.827207,0.869000,0.847589,0.826066,0.774136,0.799258,0.823424,0.826731,0.826054
800,0.427800,0.390640,0.000900,0.826130,0.827931,0.866474,0.846764,0.823642,0.775932,0.799075,0.822920,0.826130,0.825515
900,0.428900,0.392939,0.000900,0.818727,0.867850,0.793937,0.829250,0.768169,0.849573,0.806823,0.818036,0.818727,0.819257
1000,0.419000,0.375042,0.000900,0.835934,0.837426,0.873692,0.855175,0.833887,0.788954,0.810798,0.832987,0.835934,0.835402


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning:

# **Evaluate Fine-Tuned Model on Train, Dev, and Test Datasets**

In [20]:
# Split: Train, Dev, or Test
def generate_evaluation_results(split):
    dataset = None
    if split == "train":
        dataset = train_hf_dataset
    elif split == "dev" or split == "validation" or split == "val":
        dataset = dev_hf_dataset
    elif split == "test":
        dataset = test_hf_dataset
    
    results = trainer.evaluate(eval_dataset=dataset, metric_key_prefix=split)
    df_results = pd.DataFrame([results])
    df_results.to_csv(f"Milestone #2 Fine-Tuned BERT Baseline {split} Results.csv", index=False)
    print(f"Saved {split} evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline {split} Results.csv")

# Generate Evaluation Results on Train, Dev, and Test Splits
generate_evaluation_results("train")
generate_evaluation_results("dev")
generate_evaluation_results("test")

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved train evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline train Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved dev evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline dev Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved test evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline test Results.csv
